In [2]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point,Polygon
import csv
import random
from geopandas.tools import sjoin
import multiprocessing
import time
from pathos.multiprocessing import ProcessingPool as Pool
from prettytable import PrettyTable

In [2]:
!python --version
import platform
print(platform.platform())
print("cpu cores: {0}".format(multiprocessing.cpu_count()))

Python 3.7.3
Darwin-18.6.0-x86_64-i386-64bit
cpu cores: 4


In [3]:
lat = []
lon = []
data_points = 100000
for i in range(data_points):
    x = random.uniform(-90,90)
    lat.append(x)
    
    
for i in range(data_points):
    x = random.uniform(-180,180)
    lon.append(x)

df_cord = pd.DataFrame(lon , columns = ['Lon'])
df_cord['Lat'] = lat
geometry = [Point(xy) for xy in zip(df_cord.Lon, df_cord.Lat)]
df_cord = df_cord.drop(['Lon' , 'Lat' ], axis=1)
crs = {'init': 'epsg:4326'}
gdf = geopandas.GeoDataFrame(df_cord, crs=crs, geometry=geometry)

In [4]:
print(gdf.shape)

(100000, 1)


In [5]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
# ax = world.plot( color='white', edgecolor='black' ,figsize = (40,30))
# gdf.plot(ax=ax, color='red')
# plt.title("World map showing location of given coordinate")
# print(world.head(3))
# plt.show()

In [6]:
df_new = world[['continent' , 'name' , 'geometry']]

In [7]:
def computation(data_split):
    for index,row in data_split.iterrows():
        filter = df_new['geometry'].contains(row['geometry'])
        x = df_new.loc[filter]
        value_c = x['name'].to_string(index = False)
        data_split.loc[index, 'Country'] = value_c if value_c != 'Series([], )' else np.NaN
    return data_split

In [8]:
#computation(gdf)

In [9]:
start_time = time.time()  

cores = 16
partitions = 16


data_split = np.array_split(gdf, cores, axis=0)
pool = Pool(cores)
for i in range(len(data_split)):
    print(data_split[i].shape)
df_out = np.vstack(pool.map(computation, data_split))
pool.close()
pool.join()
pool.clear()
print("--- %s seconds ---" % (time.time() - start_time))

(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
(6250, 1)
--- 213.08127999305725 seconds ---


In [10]:
op  = pd.DataFrame(data = df_out)
print(op)

                                                   0                 1
0       POINT (-99.90811557000129 57.01409711852594)            Canada
1       POINT (-160.2887368042639 77.66883577638831)               NaN
2        POINT (175.9016256556254 40.52129850108327)               NaN
3      POINT (-111.4273966019301 -89.62595891533937)        Antarctica
4       POINT (34.71782639384764 -65.62833163578861)               NaN
5        POINT (111.9882467757029 -25.5673409700282)               NaN
6       POINT (24.45843345527911 -2.480037226276963)   Dem. Rep. Congo
7       POINT (-174.9185150908384 11.43981517633384)               NaN
8       POINT (-134.0419217231101 28.01856448458116)               NaN
9        POINT (117.3965502397749 36.02321487574385)             China
10       POINT (23.5020595733254 -26.52499719262659)      South Africa
11       POINT (112.1719725145896 70.37431381937975)            Russia
12     POINT (-53.61506528945073 -27.63367912752224)            Brazil
13    

In [7]:
x = PrettyTable()

x.field_names = ["S.No." ," Number of Partitions", "Data Points per partition", "Time taken to run(minutes)"]
x.add_row([0, 0, 1000000, 155])
x.add_row([1, 4, 250000, 55])
x.add_row([2, 8, 125000, 40])
x.add_row([3, 12, 83333, 36])                 
x.add_row([4, 16, 6250, 35.5])

In [9]:
print("For 1 million data points")
print(x)

For 1 million data points
+-------+-----------------------+---------------------------+----------------------------+
| S.No. |  Number of Partitions | Data Points per partition | Time taken to run(minutes) |
+-------+-----------------------+---------------------------+----------------------------+
|   0   |           0           |          1000000          |            155             |
|   1   |           4           |           250000          |             55             |
|   2   |           8           |           125000          |             40             |
|   3   |           12          |           83333           |             36             |
|   4   |           16          |            6250           |            35.5            |
+-------+-----------------------+---------------------------+----------------------------+
